<a href="https://colab.research.google.com/github/Matthew-Gallardo/DF-Detection/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before running the file Upload all your data set on your goole drive in a zip format

In [3]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set harware accelarotor as GPU)
#download and unzip the data from google drive Colab environment
from google_drive_downloader import GoogleDriveDownloader as gdd
#use only file id of the link
#Note: Below link is just an example, Not an actual link. Actual Links are in ReadMe file
#https://drive.google.com/file/d/1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07/view?usp=sharing
#https://drive.google.com/drive/folders/19wnm4BTrqrGFHBtr5X31-1nPgVN9-k6m?usp=sharing
url = '1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07'
gdd.download_file_from_google_drive(file_id = url,dest_path='./data.zip',unzip=True)

In [8]:
import json
import glob
import numpy as np
import cv2
import copy

# set the path to the directories containing the video files
dir1 = '/content/drive/MyDrive/Deepfake Dataset/Celeb-real'
dir2 = '/content/drive/MyDrive/Deepfake Dataset/Celeb-synthesis'

# use glob to find all MP4 files in both directories
video_files = glob.glob(dir1 + '/*.mp4') + glob.glob(dir2 + '/*.mp4')

frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150:
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

print("Frames per video:", frame_count)
print("Total number of videos:", len(frame_count))
print('Average frames per video:', np.mean(frame_count))


Frames per video: [466, 435, 420, 477, 315, 530, 326, 272, 352, 412, 489, 483, 425, 330, 387, 461, 611, 357, 314, 451, 364, 464, 459, 570, 381, 318, 291, 519, 317, 224, 303, 492, 390, 459, 555, 397, 372, 332, 312, 314, 326, 327, 303, 497, 491, 340, 401, 630, 520, 378, 557, 322, 409, 306, 510, 535, 428, 566, 311, 363, 398, 161, 502, 380, 740, 401, 529, 415, 409, 350, 410, 516, 448, 422, 307, 172, 350, 534, 345, 477, 276, 270, 298, 349, 215, 469, 328, 333, 308, 380, 458, 280, 294, 325, 505, 239, 380, 467, 456, 316, 470, 295, 372, 313, 379, 269, 458, 214, 364, 361, 337, 439, 514, 304, 372, 339, 400, 361, 376, 437, 444, 383, 316, 305, 322, 351, 320, 315, 371, 459, 446, 398, 441, 479, 400, 284, 517, 274, 327, 499, 323, 525, 201, 334, 247, 371, 452, 373, 376, 457, 458, 445, 308, 459, 362, 458, 309, 499, 458, 479, 316, 247, 305, 534, 557, 315, 489, 470, 458, 274, 201, 323, 315, 470, 529, 469, 387, 459, 362, 466, 312, 456, 456, 280, 318, 461, 420, 364, 409, 415, 201, 529, 442, 464, 380, 294, 4

In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path) 
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image
!pip3 install face_recognition
!mkdir '/content/drive/My Drive/FF_REAL_Face_only_data'
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/FF_REAL_Face_only_data/')